In [ ]:
import pandas as pd
#use to connect database
import sqlalchemy as sal
engine=sal.create_engine('mssql://LAPTOP-45\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()

In [ ]:
def extract():
    #read text file
    df_product=pd.read_csv('products.txt')
    #read sql table
    df_product_db=pd.read_sql_query('select *from product_p1',conn)
    return df_product,df_product_db
    
def transform(df_product,df_product_db):
    #here ijoin both table
    df_merge=pd.merge(df_product,df_product_db,how='left',on='product_id')
    #check which column null value
    df_insert=df_merge[df_merge['product_name_y'].isna()]
    #here (: mean all rows )
    df_insert_final=df_insert.iloc[:,0:3]
    #here i asign column
    df_insert_final.columns=df_product_db.columns
    
    df_update=df_merge[df_merge['product_name_y'].notna()]
    df_update_final=df_update.iloc[:,0:3]
    df_update_final.columns=df_product_db.columns
    return df_insert_final,df_update_final

def load_staging(df_update_final):
     df_update_final.to_sql('products_stg',con=conn , index=False , if_exists = 'replace')
     conn.commit()

def updates():
    query = sal.text("update product_p1 set price = products_stg.price, product_name=products_stg.product_name from products_stg where product_p1.product_id=products_stg.product_id")
    p = conn.execute(query)
    conn.commit()


def load_insert(df_insert):
    df_insert.to_sql('product_p1',con=conn , index=False , if_exists = 'append')
    conn.commit()
    

In [ ]:
df_product,df_product_db=extract()
df_insert_final,df_update_final=transform(df_product,df_product_db)
load_insert(df_insert_final)
load_staging(df_update_final)
updates()